In [1]:
import dask.dataframe as dd
from dask_ml.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score
from joblib import dump,load
import numpy as np
import time

# load the classification model

In [29]:
import matplotlib.pyplot as plt
import pandas as pd

#load data and feature_important_axis1
datapath=r'../Dataanalyse/axis2_demo_tablepart1_0.csv'
import json
with open('feature_important_axis1.json') as f:
    feature_important_axis1 = json.load(f)

feature_important_axis1
df = pd.read_csv(datapath)
print(len(df))
df.drop_duplicates(subset=[df.columns[0]], inplace=True)
df["date"] = pd.to_datetime(df.Timestamp, unit='s')

features=["date",'Schadensklasse']+feature_important_axis1

print(features)
data_selected = df[features]
data_selected.head()

10000
['date', 'Schadensklasse', 'Effektivwert_z', 'Median_z', 'Mittlere_Absolute_Abweichung_z', 'Mittelwert_z', 'Median_y', 'Effektivwert_y', 'Effektivwert_x', 'Variance_z', 'Mittelwert_x', 'Standardabweichung_z']


,date,Schadensklasse,Effektivwert_z,Median_z,Mittlere_Absolute_Abweichung_z,Mittelwert_z,Median_y,Effektivwert_y,Effektivwert_x,Variance_z,Mittelwert_x,Standardabweichung_z
0,2020-11-04 11:40:00,2,15896.6,15872,350.131,15889.7,-512,571.115,657.526,220962,569.600,469.831
1,2020-11-04 11:40:00,2,15887.1,15872,363.095,15879.7,-512,571.746,627.592,235057,545.792,484.584
2,2020-11-04 11:40:00,2,15893.9,15872,395.262,15885.8,-512,566.507,651.770,256573,565.248,506.277
3,2020-11-04 11:40:00,2,15927.7,15872,389.710,15920.1,-512,568.931,653.578,241982,566.784,491.670
4,2020-11-04 11:40:00,2,15925.5,15872,414.196,15917.1,-512,586.402,661.006,268509,567.040,517.919


In [34]:
# define the length
alltime=np.unique(data_selected.date)
alltime
after=[]
for x in alltime:
    length=len(data_selected[data_selected["date"]==x])
    if length >=2000:
        after.append(x)
        print('length of batches: %d',length)
print('='*30)


length of batches: %d 2201
length of batches: %d 2348
length of batches: %d 2692


In [37]:
# only select the last batch 
for time in after:
    finaldata=data_selected[data_selected["date"]==time]

In [87]:
test=finaldata[feature_important_axis1].to_numpy()
test.shape

(2692, 10)

In [46]:
# load the axis1 model with 10 best features
model = load('10_best_xgboost_modeltestaxis1.joblib')
res=model.predict(test)
print(len(res))
res

FileNotFoundError: [Errno 2] No such file or directory: '../10_best_xgboost_modeltestaxis1.joblib'

In [79]:
truedata=data_selected[data_selected["date"]==time]
true=truedata.Schadensklasse.to_list()
true=np.array(true)
print(len(true))
true

In [85]:
import numpy as np

# check the length of two arrays
if len(res) != len(true):
    raise ValueError("两个数组的长度不一致。")

# compare two arrays
similarity_array = res == true
print(similarity_array)
# calculate the number of similar elements
similar_count = np.sum(similarity_array)
print(similar_count)
# calculate the similarity ratio
similarity_ratio = similar_count / len(res)

print(f"相同元素的比例: {similarity_ratio:.2f}")


2692


array([2, 2, 2, ..., 2, 2, 2])

# load the time series model

In [182]:
class Configs:
    pred_len = 200
    output_attention = False
    enc_in = 10
    d_model = 512
    embed = 'timeF'
    freq = 's'
    dropout = 0.05
    e_layers = 2
    c_out = 10
    seq_len=200
    label_len=100
    pred_len=200
    step_size=200

configs = Configs()

In [183]:
# load the test dataset numpy array length 2000
import numpy as np  
import os
import pandas as pd
from joblib import dump,load

datapath=r'Dataanalyse/all_data_axis1.csv'
test_df = pd.read_csv(datapath)
test_df.drop_duplicates(subset=[test_df.columns[0]], inplace=True)
df = test_df.iloc[90000:200000].copy()
df.shape

(110000, 31)

In [184]:
import json
with open('syntheic_data/feature_important_axis1.json') as f:
    feature_important_axis1 = json.load(f)
df["date"] = pd.to_datetime(df.Timestamp, unit='s')

features=["date",'Schadensklasse']+feature_important_axis1
data_selected = df[features]
data_selected.shape

(110000, 12)

In [185]:
data=data_selected
alltime=np.unique(data.date)
after=[]
for x in alltime:
    length=len(data[data["date"]==x])
    damage=np.unique(data[data["date"]==x].Schadensklasse)
    # if length >10000 and len(damage)>=2:

    if length >=2000:
        after.append(x)
        print('length of batches: %d',length)
print('='*30)


length of batches: %d 8600
length of batches: %d 13834
length of batches: %d 17411
length of batches: %d 17419
length of batches: %d 17358
length of batches: %d 17363
length of batches: %d 16486


In [186]:
time=after[0]
time

numpy.datetime64('2021-01-15T03:06:40.000000000')

# try single 200

In [204]:
from datetime import timedelta
from timefeatures import time_features
import torch
from torch.utils.data import Dataset, DataLoader,IterableDataset,ConcatDataset,TensorDataset

# load the scaler
import pathlib as Path
import pickle

Scaler_file = r'D:\result\second\axis1_LSTM_Custom_axis1_ftM_sl200_ll100_pl200_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_test_0\data\scaler.pkl'
with open(Scaler_file, 'rb') as f:
    full_scaler = pickle.load(f)

finaldata=data[data["date"]==time]
finaldata=finaldata[:200]
finaldata.shape
zeros_df = pd.DataFrame(np.zeros((200, 12)), columns=finaldata.columns)
finaldata = pd.concat([finaldata, zeros_df], ignore_index=True)
print(finaldata.shape)  # 应该输出 (400, 12)
start_date = finaldata['date'].iloc[0]
# 生成一个时间序列，从起始日期开始，每秒递增，长度与data相同
time_series = [start_date + timedelta(seconds=i) for i in range(len(finaldata))]

# 将生成的时间序列赋值回data的'date'列
finaldata.loc[:, 'date'] = time_series

dataset = finaldata
print('length of the testing dataset after times : %d', len(dataset))
df_stamp = dataset['date']
df_stamp.date = pd.to_datetime(df_stamp)



data_stamp = time_features(df_stamp, timeenc=1, freq='s')
data_stamp = torch.FloatTensor(data_stamp)
print(data_stamp.shape)

dataset = dataset.copy()

#dataset.drop(['date',"Schadensklasse"], axis=1, inplace=True)
dataset = pd.DataFrame(dataset, columns=finaldata.columns.drop(['date', 'Schadensklasse']))
dataset = full_scaler.transform(dataset)
dataset = torch.FloatTensor(dataset)

(400, 12)
length of the testing dataset after times : %d 400
torch.Size([400, 6])


In [194]:
len(dataset) 
for x in range(0,1,200):
    print(x)

0


In [216]:
dataset[200:201]

tensor([[-0.0267,  0.5010,  0.0000,  0.5010,  0.0590,  0.0000, -0.0099,  0.0000,
          0.5005,  0.0000]])

In [229]:
# 构造样本
samples = []
Dte_samples=[]
count = 0
for index in range(0, len(dataset) - configs.seq_len - configs.pred_len + 1, configs.step_size):
    # train_x, x_mark, train_y, y_mark
    s_begin = index
    s_end = s_begin + configs.seq_len # 200
    r_begin = s_end - configs.label_len # 100
    r_end = r_begin + configs.label_len + configs.pred_len #300
    seq_x = dataset[s_begin:s_end] # 0-200
    seq_y = dataset[r_begin:r_end] # 100-300
    seq_x_mark = data_stamp[s_begin:s_end] # 0-200
    seq_y_mark = data_stamp[r_begin:r_end] # 100-300
    samples.append((seq_x, seq_y, seq_x_mark, seq_y_mark))
    count += 1


print('number of dataset in one batch: %d', count)
samples = DataLoader(dataset=samples, batch_size=32, shuffle=False, num_workers=0, drop_last=False, pin_memory=True)
print('='*30)
Dte_samples.append(samples)
print('number of batches: %d', len(samples))


number of dataset in one batch: %d 1
number of batches: %d 1


In [234]:
class CustomDataset(Dataset):
    def __init__(self, dataloader):
        self.dataloader = dataloader
        self.dataset = dataloader.dataset
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx]


Dte_datasets = [CustomDataset(dataloader) for dataloader in Dte_samples]
Dte_combined_dataset = ConcatDataset(Dte_datasets)
Dte_combined_dataloader = DataLoader(Dte_combined_dataset, batch_size=1, shuffle=False)

In [235]:
import torch
import torch.nn as nn
import pandas as pd
from datetime import timedelta
from Embed import DataEmbedding
from torch.utils.data import Dataset, DataLoader, ConcatDataset


# Define your model architecture
class Model(nn.Module):
    def __init__(self, configs):
        super(Model, self).__init__()
        self.pred_len = configs.pred_len
        self.output_attention = configs.output_attention

        # Embedding layer
        self.enc_embedding = DataEmbedding(configs.enc_in, configs.d_model, configs.embed, configs.freq, configs.dropout)
        
        # LSTM Layer
        self.lstm = nn.LSTM(
            input_size=configs.d_model,
            hidden_size=configs.d_model,
            num_layers=configs.e_layers,
            bidirectional=False,
            batch_first=True
        )
        
        # Projection layer
        num_directions = 1
        self.projection = nn.Linear(configs.d_model * num_directions, configs.c_out)

    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec, enc_self_mask=None):
        x_enc = self.enc_embedding(x_enc, x_mark_enc)
        lstm_out, _ = self.lstm(x_enc)
        lstm_out = self.projection(lstm_out)

        if self.output_attention:
            return lstm_out[:, -self.pred_len:, :], None
        else:
            return lstm_out[:, -self.pred_len:, :]

In [242]:
from models import Informer, Autoformer, Transformer, Reformer,LSTM,GRU


In [236]:
import os
import torch
def acquire_device():
    device = torch.device('cpu')
    print('Use CPU')
    return device
device = acquire_device()

model_path = r"D:\result\second\axis2_LSTM_Custom_axis2_ftM_sl200_ll100_pl200_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_test_0\checkpoints\checkpoint.pth"
model = Model(configs).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()


def predict( batch_x, batch_y, batch_x_mark, batch_y_mark):
    # decoder input
    dec_inp = torch.zeros_like(batch_y[:, -configs.pred_len:, :]).float()
    dec_inp = torch.cat([batch_y[:, :configs.label_len, :], dec_inp], dim=1).float().to(device)
    # encoder - decoder

    def _run_model():
        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        return outputs
    outputs = _run_model()

    f_dim = 0
    outputs = outputs[:, -configs.pred_len:, f_dim:]
    batch_y = batch_y[:, -configs.pred_len:, f_dim:].to(device)

    return outputs, batch_y

Use CPU


In [237]:
for i,(batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(Dte_combined_dataloader):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float()

        batch_x_mark = batch_x_mark.float().to(device)
        batch_y_mark = batch_y_mark.float().to(device)
        #print('batch_x shape',batch_x.shape)

        outputs, batch_y = predict(batch_x, batch_y, batch_x_mark, batch_y_mark)
        print(outputs.shape)

torch.Size([1, 200, 10])


In [239]:
# 确保张量在 CPU 上
outputs_cpu = outputs[0].cpu()
# 转换为 NumPy 数组
outputs_numpy = outputs_cpu.detach().numpy()
# 打印 NumPy 数组的形状以确认转换
print(outputs_numpy.shape)
outputs_numpy=full_scaler.inverse_transform(outputs_numpy)

res=outputs_numpy#[1,:].reshape(1,10)
res.shape
# load the axis1 model with 10 best features
model = load('10_best_xgboost_modeltestaxis1.joblib')
res=model.predict(res)
res


(200, 10)


array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=int64)

# multiple batch

In [93]:
from datetime import timedelta
from timefeatures import time_features
import torch
from torch.utils.data import Dataset, DataLoader,IterableDataset,ConcatDataset,TensorDataset

# load the scaler
import pathlib as Path
import pickle
Scaler_file = r'D:\result\second\axis1_LSTM_Custom_axis1_ftM_sl200_ll100_pl200_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_test_0\data\scaler.pkl'
with open(Scaler_file, 'rb') as f:
    full_scaler = pickle.load(f)


Dte_samples=[]
finaldata=data[data["date"]==time]
start_date = finaldata['date'].iloc[0]
# 生成一个时间序列，从起始日期开始，每秒递增，长度与data相同
time_series = [start_date + timedelta(seconds=i) for i in range(len(finaldata))]

# 将生成的时间序列赋值回data的'date'列
finaldata.loc[:, 'date'] = time_series

dataset = finaldata
print('length of the testing dataset after times 0.2: %d', len(dataset))

df_stamp = dataset['date']
df_stamp.date = pd.to_datetime(df_stamp)
data_stamp = time_features(df_stamp, timeenc=1, freq='s')
data_stamp = torch.FloatTensor(data_stamp)
print(data_stamp.shape)

dataset = dataset.copy()

#dataset.drop(['date',"Schadensklasse"], axis=1, inplace=True)
dataset = pd.DataFrame(dataset, columns=finaldata.columns.drop(['date', 'Schadensklasse']))

dataset = full_scaler.transform(dataset)
dataset = torch.FloatTensor(dataset)
# 构造样本
samples = []
count = 0
for index in range(0, len(dataset) - configs.seq_len - configs.pred_len + 1, configs.step_size):
    # train_x, x_mark, train_y, y_mark
    s_begin = index
    s_end = s_begin + configs.seq_len
    r_begin = s_end - configs.label_len
    r_end = r_begin + configs.label_len + configs.pred_len
    seq_x = dataset[s_begin:s_end]
    seq_y = dataset[r_begin:r_end]
    seq_x_mark = data_stamp[s_begin:s_end]
    seq_y_mark = data_stamp[r_begin:r_end]
    samples.append((seq_x, seq_y, seq_x_mark, seq_y_mark))
    count += 1


print('number of dataset in one batch: %d', count)
samples = DataLoader(dataset=samples, batch_size=32, shuffle=False, num_workers=0, drop_last=False, pin_memory=True)
print('='*30)
Dte_samples.append(samples)
print('number of batches: %d', len(samples))


length of the testing dataset after times 0.2: %d 8600
torch.Size([8600, 6])
number of dataset in one batch: %d 42
number of batches: %d 2


In [90]:
for x,y,z,p in samples:
    print(x.shape)
    print(y.shape)
    print(z.shape)
    print(p.shape)
    break


torch.Size([32, 200, 10])
torch.Size([32, 300, 10])
torch.Size([32, 200, 6])
torch.Size([32, 300, 6])


In [ ]:
# from datetime import timedelta
# from timefeatures import time_features
# import torch
# from torch.utils.data import Dataset, DataLoader,IterableDataset,ConcatDataset,TensorDataset

# # load the scaler
# import pathlib as Path
# import pickle
# Scaler_file = r'D:\result\second\axis1_LSTM_Custom_axis1_ftM_sl200_ll100_pl200_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_test_0\data\scaler.pkl'
# with open(Scaler_file, 'rb') as f:
#     full_scaler = pickle.load(f)


# Dte_samples=[]
# for time in after:
#     finaldata=data[data["date"]==time]
#     start_date = finaldata['date'].iloc[0]
#     # 生成一个时间序列，从起始日期开始，每秒递增，长度与data相同
#     time_series = [start_date + timedelta(seconds=i) for i in range(len(finaldata))]

#     # 将生成的时间序列赋值回data的'date'列
#     finaldata.loc[:, 'date'] = time_series

#     dataset = finaldata
#     print('length of the testing dataset after times 0.2: %d', len(dataset))

#     df_stamp = dataset['date']
#     df_stamp.date = pd.to_datetime(df_stamp)
#     data_stamp = time_features(df_stamp, timeenc=1, freq='s')
#     data_stamp = torch.FloatTensor(data_stamp)
#     print(data_stamp.shape)

#     dataset = dataset.copy()

#     dataset.drop(['date',"Schadensklasse"], axis=1, inplace=True)
#     dataset = pd.DataFrame(dataset, columns=finaldata.columns.drop(['date', 'Schadensklasse']))

#     dataset = full_scaler.transform(dataset)
#     dataset = torch.FloatTensor(dataset)
#     # 构造样本
#     samples = []
#     count = 0
#     for index in range(0, len(dataset) - configs.seq_len - configs.pred_len + 1, configs.step_size):
#         # train_x, x_mark, train_y, y_mark
#         s_begin = index
#         s_end = s_begin + configs.seq_len
#         r_begin = s_end - configs.label_len
#         r_end = r_begin + configs.label_len + configs.pred_len
#         seq_x = dataset[s_begin:s_end]
#         seq_y = dataset[r_begin:r_end]
#         seq_x_mark = data_stamp[s_begin:s_end]
#         seq_y_mark = data_stamp[r_begin:r_end]
#         samples.append((seq_x, seq_y, seq_x_mark, seq_y_mark))
#         count += 1


#     print('number of dataset in one batch: %d', count)
#     samples = DataLoader(dataset=samples, batch_size=32, shuffle=False, num_workers=0, drop_last=False, pin_memory=True)
#     print('='*30)
#     Dte_samples.append(samples)
# print('number of batches: %d', len(Dte_samples))

In [100]:
class CustomDataset(Dataset):
    def __init__(self, dataloader):
        self.dataloader = dataloader
        self.dataset = dataloader.dataset
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx]


Dte_datasets = [CustomDataset(dataloader) for dataloader in Dte_samples]
Dte_combined_dataset = ConcatDataset(Dte_datasets)
Dte_combined_dataloader = DataLoader(Dte_combined_dataset, batch_size=1, shuffle=False)



# Single batch

In [101]:
for i,(batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(Dte_combined_dataloader):
    print(batch_x.shape)
    print(i)

torch.Size([1, 200, 10])
0
torch.Size([1, 200, 10])
1
torch.Size([1, 200, 10])
2
torch.Size([1, 200, 10])
3
torch.Size([1, 200, 10])
4
torch.Size([1, 200, 10])
5
torch.Size([1, 200, 10])
6
torch.Size([1, 200, 10])
7
torch.Size([1, 200, 10])
8
torch.Size([1, 200, 10])
9
torch.Size([1, 200, 10])
10
torch.Size([1, 200, 10])
11
torch.Size([1, 200, 10])
12
torch.Size([1, 200, 10])
13
torch.Size([1, 200, 10])
14
torch.Size([1, 200, 10])
15
torch.Size([1, 200, 10])
16
torch.Size([1, 200, 10])
17
torch.Size([1, 200, 10])
18
torch.Size([1, 200, 10])
19
torch.Size([1, 200, 10])
20
torch.Size([1, 200, 10])
21
torch.Size([1, 200, 10])
22
torch.Size([1, 200, 10])
23
torch.Size([1, 200, 10])
24
torch.Size([1, 200, 10])
25
torch.Size([1, 200, 10])
26
torch.Size([1, 200, 10])
27
torch.Size([1, 200, 10])
28
torch.Size([1, 200, 10])
29
torch.Size([1, 200, 10])
30
torch.Size([1, 200, 10])
31
torch.Size([1, 200, 10])
32
torch.Size([1, 200, 10])
33
torch.Size([1, 200, 10])
34
torch.Size([1, 200, 10])
35
to

In [102]:
import torch
import torch.nn as nn
import pandas as pd
from datetime import timedelta
from Embed import DataEmbedding
from torch.utils.data import Dataset, DataLoader, ConcatDataset


# Define your model architecture
class Model(nn.Module):
    def __init__(self, configs):
        super(Model, self).__init__()
        self.pred_len = configs.pred_len
        self.output_attention = configs.output_attention

        # Embedding layer
        self.enc_embedding = DataEmbedding(configs.enc_in, configs.d_model, configs.embed, configs.freq, configs.dropout)
        
        # LSTM Layer
        self.lstm = nn.LSTM(
            input_size=configs.d_model,
            hidden_size=configs.d_model,
            num_layers=configs.e_layers,
            bidirectional=False,
            batch_first=True
        )
        
        # Projection layer
        num_directions = 1
        self.projection = nn.Linear(configs.d_model * num_directions, configs.c_out)

    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec, enc_self_mask=None):
        x_enc = self.enc_embedding(x_enc, x_mark_enc)
        lstm_out, _ = self.lstm(x_enc)
        lstm_out = self.projection(lstm_out)

        if self.output_attention:
            return lstm_out[:, -self.pred_len:, :], None
        else:
            return lstm_out[:, -self.pred_len:, :]

In [103]:
import os
import torch
def acquire_device():
    device = torch.device('cpu')
    print('Use CPU')
    return device
device = acquire_device()
import torch

model_path = r"D:\result\second\axis2_LSTM_Custom_axis2_ftM_sl200_ll100_pl200_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_test_0\checkpoints\checkpoint.pth"
model = Model(configs).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()


def predict( batch_x, batch_y, batch_x_mark, batch_y_mark):
    # decoder input
    dec_inp = torch.zeros_like(batch_y[:, -200:, :]).float()
    dec_inp = torch.cat([batch_y[:, :100, :], dec_inp], dim=1).float().to(device)
    # encoder - decoder

    def _run_model():
        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        return outputs
    outputs = _run_model()

    f_dim = 0
    outputs = outputs[:, -200:, f_dim:]
    batch_y = batch_y[:, -200:, f_dim:].to(device)

    return outputs, batch_y

Use CPU


In [104]:
for i,(batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(Dte_combined_dataloader):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float()

        batch_x_mark = batch_x_mark.float().to(device)
        batch_y_mark = batch_y_mark.float().to(device)
        #print('batch_x shape',batch_x.shape)

        outputs, batch_y = predict(batch_x, batch_y, batch_x_mark, batch_y_mark)
        print(outputs.shape)

torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])
torch.Size([1, 200, 10])


In [108]:
outputs[0].shape

torch.Size([200, 10])

In [123]:
# 确保张量在 CPU 上
outputs_cpu = outputs[0].cpu()
# 转换为 NumPy 数组
outputs_numpy = outputs_cpu.detach().numpy()
# 打印 NumPy 数组的形状以确认转换
print(outputs_numpy.shape)
outputs_numpy=full_scaler.inverse_transform(outputs_numpy)

res=outputs_numpy[1,:].reshape(1,10)
res.shape
# load the axis1 model with 10 best features
model = load('10_best_xgboost_modeltestaxis1.joblib')
res=model.predict(res)
res


(200, 10)


In [139]:
outputs_numpy.shape

(200, 10)

In [141]:
res=outputs_numpy[1,:].reshape(1,10)
res.shape

(1, 10)

In [142]:
# load the axis1 model with 10 best features
model = load('10_best_xgboost_modeltestaxis1.joblib')
res=model.predict(res)
res

array([2], dtype=int64)